In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import HTMLResponse
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
df_movies= pd.read_csv(r'C:\Users\flori\Desktop\DATA SCIENCE\LABS\Trabajos\Proyecto_Indiv_1_MLO_Henry\ETL\A-ETLconsignas\MoviesSMALL.csv')
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44936 entries, 0 to 44935
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             44936 non-null  float64
 1   id                 44936 non-null  int64  
 2   original_language  44926 non-null  object 
 3   overview           44029 non-null  object 
 4   popularity         44936 non-null  float64
 5   release_date       44936 non-null  object 
 6   revenue            44936 non-null  float64
 7   runtime            44696 non-null  float64
 8   title              44936 non-null  object 
 9   release_year       44936 non-null  float64
 10  return             10879 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 3.8+ MB


In [3]:
df_top_5=pd.read_csv('Top15kpelis.csv')
filters=pd.read_csv('filters.csv')
df_concat3=pd.read_csv('FuncionML.csv')

In [4]:
df_top_5.isnull().mean()

id            0.000000
overview      0.002933
popularity    0.000000
title         0.000000
dtype: float64

In [5]:
df_top_5['overview'].fillna('', inplace=True) 

In [6]:
df_top_4= df_top_5.head(5000)

In [7]:
df_top_4.to_csv('Funcion4milpelis.csv', index=False)

In [8]:
filters.isnull().mean()

overview       0.002933
title          0.000000
character      0.002667
name           0.001467
name_genres    0.001200
metadatos      0.000000
dtype: float64

In [9]:
filters.fillna('', inplace=True) 

In [10]:
filters2= filters.head(4000)

In [11]:
filters2.to_csv('filters2.csv', index=False)

In [14]:

indices=pd.Series(filters2.index, index=filters2['title'])

def recomendacion_peli_top(title):
    '''Ingresa un nombre de película y devuelve una lista de películas similares basadas en sus resúmenes'''
    
    # instancia de CountVectorizer para convertir los textos en vectores numéricos 
    count = CountVectorizer(stop_words='english') # palabras vacias o comunes en el idioma indicado como english .
    count_matrix = count.fit_transform(filters2['metadatos']) # utilizamos la columna overviwe de nuestro dataframe df_top_5., y matriziamos el conteo de cada palabra
    
    # # Calcularemos la similitud del coseno entre los vectores que representan los overview de las películas en df_top_5.
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    
    # Convertir el título a minúsculas y eliminar espacios en blanco
    title = title.replace(' ', '').lower() 
    
    # índice de la película en el df_top_5
    if title not in indices: # esto verifica si el nombre ingresado se encuentra en la columna title
        return {'Mensaje': 'Película no encontrada'} # si no la encuentra sale este mensaje
    
    idx = df_top_4[df_top_4['title'].str.replace(' ', '').str.lower() == title].index[0] #  busca el índice correspondiente de la película en el DataFrame df_top_5
    
    # nos indica los puntajes de similitud y los ordena.
    sim_scores = list(enumerate(cosine_sim[idx])) # aqui esta obteniendo los puntajes de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # posteriomente los estamos ordenando. El lambda usa como entrada (x) y devuelve el valor de similitud (x[1])
    # ponemo el reverse true porque ordena en orden descendente según el valor de similitud, valga la redundancia.



    #Para que devuelva solamente las pelis unicamente que indiquemos, si elimino esta linea devuelve una lista mucho mayor de peliculas similares.
    sim_scores = sim_scores[1:6] #[1:6] para que devuelva solamente 5
    
    # índices de las películas similares (excluyendo la película consultada)
    similar_movie_indices = [i[0] for i in sim_scores if i[0] != idx]
    # Tomaremos cada elemento 'i' en la lista 'sim_scores'.
    # extrae los indices creados en sim_scores y crea una lista de índices (i[0]) solamente si el índice no es igual al índice de la película consultada (idx). 
    # Esto excluye el índice de la película consultada de la lista de índices de películas similares, != idx] esto indica eso, es decir que no es igual al indice de la peli indicada.
    
    # Obtener la lista de títulos de películas similares
    similar_movie_titles = df_top_4['title'].iloc[similar_movie_indices].tolist()
    
    return similar_movie_titles 

In [15]:
recomendacion_peli_top('toy story')

['Toy Story 2',
 'Toy Story 3',
 'Small Fry',
 'Hawaiian Vacation',
 'In a Heartbeat']

In [16]:
recomendacion_peli_top('wonder woman')

['Fantastic Beasts and Where to Find Them',
 'Pokémon: Spell of the Unknown',
 'Highlander',
 'The Last Legion',
 'Doctor Strange']